In [4]:
import json
import torch
import json
import os
from LayoutDM import CLDM
from torch.utils.data import Dataset, DataLoader
from diffusers import DDPMScheduler
from PIL import Image
import numpy as np

In [5]:
diffusion = DDPMScheduler()

In [6]:
device = f"cuda:{5}" if torch.cuda.is_available() else "cpu"

## Load Model with Seperate Param dict

In [7]:
save_path ='/workspace/joonsm/City_Layout/log_dir/FPN[resnet50]_freeze/checkpoints/checkpoint-0/pytorch_model.bin'

model = CLDM(use_temp=True)
model.load_state_dict(torch.load(save_path))
model = model.to(device)
model.eval()

RuntimeError: Error(s) in loading state_dict for CLDM:
	Missing key(s) in state_dict: "transformer.encoder.layers.0.temp_attention.q_fc.weight", "transformer.encoder.layers.0.temp_attention.q_fc.bias", "transformer.encoder.layers.0.temp_attention.k_fc.weight", "transformer.encoder.layers.0.temp_attention.k_fc.bias", "transformer.encoder.layers.0.temp_attention.v_fc.weight", "transformer.encoder.layers.0.temp_attention.v_fc.bias", "transformer.encoder.layers.0.temp_attention.out_fc.weight", "transformer.encoder.layers.0.temp_attention.out_fc.bias", "transformer.encoder.layers.0.residual3.norm.weight", "transformer.encoder.layers.0.residual3.norm.bias", "transformer.encoder.layers.1.temp_attention.q_fc.weight", "transformer.encoder.layers.1.temp_attention.q_fc.bias", "transformer.encoder.layers.1.temp_attention.k_fc.weight", "transformer.encoder.layers.1.temp_attention.k_fc.bias", "transformer.encoder.layers.1.temp_attention.v_fc.weight", "transformer.encoder.layers.1.temp_attention.v_fc

In [ ]:
model.transformer.attention.q_fc.weight

In [ ]:
a['transformer.encoder.layers.0.position_ff.fc1.bias']

In [ ]:
a= torch.load(save_path)

In [ ]:
for batch_idx, sample in enumerate(loader):
    print(f"Batch {batch_idx} SR: {sample['sr']}")
    if batch_idx > 0:
        break  # 첫 번째 배치만 확인

In [ ]:
val_path = "/nas2/lait/1000_Members/jjoonvely/CARLA/pre_seg_combined_val.json"

In [ ]:
val = ImageLayout_Val(val_path)
dataset = DataLoader(val, batch_size=len(val))
val_data = next(iter(dataset))

In [ ]:
num_sample  = val_data['box'].shape
model.eval()
noisy_batch = {'image':val_data['image'].to(device),
               'box': torch.rand(*num_sample, dtype=torch.float32, device=device)}
for i in range(1000)[::-1]:
    t = torch.tensor([i]*num_sample[0], device=device)
    with torch.no_grad():
        noise_pred = model(noisy_batch, timesteps=t)
        bbox_pred = diffusion.step(noise_pred, t[0].detach().item(),  noisy_batch['box'], return_dict=True)

        noisy_batch['box'] = bbox_pred.prev_sample


## Check the Area

In [ ]:
predicted = bbox_pred.pred_original_sample
src = val_data['sr']
src_list = []
for i in src:
    src_ = Image.open(i)
    src_list.append(src_)
box = predicted
box = box.cpu().numpy()
box = (box + 1) / 2

match_list = []

for i in range(box.shape[0]):
    img = src_list[i]
    width, height = img.size
    cx,cy,w,h = box[i]
    x = int((cx - w / 2) * width)
    y = int((cy - h / 2) * height)
    x2 = int((cx + w / 2) * width)
    y2 = int((cy + h / 2) * height)
    boxes = (x,y,x2,y2)
    crop = img.crop(boxes)
    crop = np.array(crop)
    blue_channel = crop[:,:,2]
    blue_channel_flatten = blue_channel.flatten()
    match_pixel_size = np.sum(blue_channel_flatten==128)/blue_channel_flatten.size
    match_list.append(match_pixel_size)

value = sum(match_list)/box.shape[0]